# QRC-Lab: Quantum Reservoir Computing and Risk Bounds

This lab is inspired by the paper **"Quantum Reservoir Computing and Risk Bounds"**.

### What are Risk Bounds?
In machine learning, **Risk** describes the expected error of a model. 
- **Empirical Risk**: The error on your training data.
- **True Risk**: The error on unseen data (test set).

The **Generalization Gap** is the difference between these two. The paper explores how the complexity of the quantum reservoir (number of qubits, depth) affects these bounds using tools like **Rademacher Complexity**.

### Educational Goal
In this notebook, we will visualize the **Generalization Gap** as we increase the number of qubits in our reservoir.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qrc_sim.encoders import ReuploadingEncoder
from qrc_sim.reservoirs import RandomCRotReservoir
from qrc_sim.simulator import QRCSimulator
from qrc_sim.tasks.parity import ParityTask
from qrc_sim.readout import ReadoutModel

## 1. Setup the Experiment
We will use the **Parity Task** (XOR of current and past input), which is non-linear and requires the reservoir to hold past information.

In [ ]:
task = ParityTask(length=500, delay=1)
(X_train, y_train), (X_test, y_test) = task.generate()

## 2. Scanning Reservoir Size
We will vary the number of qubits from 2 to 8 and measure the Training vs. Testing performance.

In [ ]:
qubit_range = [2, 4, 6, 8]
train_scores = []
test_scores = []

for n_q in qubit_range:
    print(f"Experimenting with {n_q} qubits...")
    
    # Highly expressive setup
    enc = ReuploadingEncoder(n_q, layers=2)
    res = RandomCRotReservoir(n_q, depth=3)
    obs = [('Z', i) for i in range(n_q)]
    
    sim = QRCSimulator(enc, res, obs, mode='reupload_k', reupload_k=3)
    
    # Get features
    f_train = sim.run_sequence(X_train)
    f_test = sim.run_sequence(X_test)
    
    # Readout
    readout = ReadoutModel(alpha=1e-5)
    readout.fit(f_train, y_train)
    
    train_scores.append(readout.score(f_train, y_train))
    test_scores.append(readout.score(f_test, y_test))

## 3. Visualizing the Generalization Gap
According to the paper, theoretical bounds on risk often scale with the dimensions of the Hilbert space. 
Let's see if we observe overfitting or improved generalization as we add qubits.

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(qubit_range, train_scores, 'o-', label='Train Score (R2)')
plt.plot(qubit_range, test_scores, 's--', label='Test Score (R2)')
plt.fill_between(qubit_range, train_scores, test_scores, alpha=0.1, color='gray', label='Generalization Gap')

plt.xlabel("Number of Qubits")
plt.ylabel("R2 Score")
plt.title("The Generalization Gap in Quantum Reservoirs")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### Discussion
1. **Expressivity**: Does adding more qubits make the Train Score hit 1.0 faster?
2. **Overfitting**: Do you see the Test Score dropping as Train Score increases? This is a sign of high **Rademacher Complexity** relative to the dataset size.
3. **Conclusion**: Risk bounds remind us that while quantum systems have massive state spaces, we must regularize our readout (e.g., higher alpha in Ridge) or provide more data to ensure the model generalizes well.